In [1]:
import numpy as np
import pandas as pd
import os
import nibabel as nib
import glob
import math
from nilearn.image import resample_img
from nilearn import plotting
get_ipython().magic(u'matplotlib inline')
import statistics
import math
import matplotlib.pyplot as plt
import time
os.chdir('/projects/hulacon/shared/neu_data/AVFP')


/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
subj_num=list(range(16,17))
print(subj_num)
#get subject name from log file names
logfile_folder = 'AffVidsMem_logfiles'
data_folder = '36p'
subj_dir = [i.split('.txt')[0].split('fmri_')[-1] for i in glob.glob(logfile_folder+'/AffVidsMem*sub*.txt')]
subj_dir.sort()
print([subj_dir[i] for i in subj_num])

search_size = 5 #edge in voxel size
vox = int((search_size-1)/2) #number of voxels from center voxel
search_size = vox*2+1
print('edge: '+str(search_size)+' voxels')
print('volume: '+str(search_size**3)+' voxels')

[16]
['sub_118']
edge: 5 voxels
volume: 125 voxels


In [3]:
log_header = ['Video_name', 'video_number', 'video_category(1-H 2-SO 3-SP)', 'novel_vs_familiar(1-N,2-Old)', 'run_number', 'video_onset', 'video_offset', 'video_duration_method1', 'video_duration_method2', 'Fear_rating_onset', 'fear_rating', 'fear_rating_RT', 'Arousal_rating_onset', 'arousal_rating', 'arousal_rating_RT', 'Valence_rating_onset', 'valence_rating', 'valence_rating_RT']
num_vid = pd.read_table(glob.glob(os.path.join(logfile_folder,'*'+subj_dir[0]+'.txt'))[0], sep=' ', header=None, names = log_header, index_col=False).shape[0]
#sample subj data img for resampling
subj_data_img = nib.load(glob.glob(os.path.join(data_folder,'sub-'+subj_dir[0][-3:]+'_run1_36p_mc_MNI_masked.nii.gz'))[0])
subj_data = subj_data_img.get_fdata()

TR=0.8
num_TR = 600
TR_series = [x*0.8 for x in range(0, num_TR)]
TR_BOLD_offset = 7
TR_video_duration = 26 #video_duration/0.8=25.6 round to 26
TR_mean_rating= 5.3*3 #rating_duration/0.8=5.3 
TR_mean_mean_ITI = 6 #ITI_duration/0.8=5.8 round to 6
TR_trial_end = TR_video_duration+TR_mean_rating+TR_mean_mean_ITI

#find the index of the closest number to K in a list
def closest(lst, K): 
    val = lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))] 
    idx = lst.index(val)
    return idx

#use index of finding value x in 2D array (b) of value x as index of the first 2 dimensions for 3D array (a)
def select3Dfrom2D(a,b,x):
    b=old_vid.astype(int)
    
    new_array = []
    for dim1 in list(set(np.where(b==x)[0])):
        new_array.append(a[dim1,np.where(b==x)[1][np.where(b==x)[0]==dim1],:])
    new_array = np.array(new_array)
    
    return new_array


In [4]:
#get grey matter mask
save_dir = '/home/chendanl/wrkdir/ISECmem_timecourse_searchlight_correlation_analysis/a'+str(search_size)+'/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
# mask = '/home/chendanl/roi/HarvardOxford/HarvardOxford-cort-maxprob-thr25-1mm.nii.gz'
# mask = '/home/chendanl/roi/HarvardOxford/HarvardOxford-sub-maxprob-thr25-1mm.nii.gz'
mask = '/home/chendanl/roi/mni_icbm152_gm_tal_nlin_asym_09a_thresh30.nii.gz'
mask_img = nib.load(mask)
mask_img = resample_img(mask_img,
                        target_affine=subj_data_img.affine,
                        target_shape=subj_data_img.shape[0:3],
                        interpolation='nearest')

#get num of voxels
# mask_location = np.where((mask_img.get_fdata()!=0) & (subj_data[:,:,:,0]!=0))
mask_location = np.where(mask_img.get_fdata()!=0)
mask_size = list(range(mask_location[0].shape[0]))

# nib.save(mask_img, '/home/chendanl/wrkdir/roi.nii.gz')
# display = plotting.plot_roi('/home/chendanl/wrkdir/roi.nii.gz', bg_img='/home/chendanl/roi/mni_icbm152_t1_tal_nlin_sym_09b_hires.nii')
# plotting.show()

In [5]:
for s in subj_num:
    subj = subj_dir[s]
    print('\n'+str(s)+': '+subj)

    #set template for result map
    result_template_brain = mask_img.get_fdata().copy()
    result_template_brain[:]=np.nan
    result_template_brain_new = result_template_brain.copy()
    result_template_brain_old = result_template_brain.copy()

    #run 1 data
    subj_run1_f = glob.glob(os.path.join(data_folder,'sub-'+subj[-3:]+'_run1_36p_mc_MNI_masked.nii.gz'))[0]
    subj_run1_img = nib.load(subj_run1_f)
    subj_run1_data = subj_run1_img.get_fdata()
    subj_run1_data[mask_img==0] = np.nan

    #run 2 data
    subj_run2_f = glob.glob(os.path.join(data_folder,'sub-'+subj[-3:]+'_run2_36p_mc_MNI_masked.nii.gz'))[0]
    subj_run2_img = nib.load(subj_run2_f)
    subj_run2_data = subj_run2_img.get_fdata()
    subj_run2_data[mask_img==0] = np.nan

    # from nilearn import datasets
    # from nilearn import surface
    # hemispheres = ['right','left']
    # pial_mesh = fsaverage['pial_' + hemi]
    # subj_run1_img_surface = surface.vol_to_surf(subj_run1_img, pial_mesh).T
    # plotting.view_surf(fsaverage.infl_right, subj_run1_img_surface[0,], bg_map=fsaverage.sulc_right)
    
    #read subject log file
    subj_log_f = glob.glob(os.path.join(logfile_folder,'AffVidsMem*'+subj+'.txt'))[0]
    subj_log_complete = pd.read_table(subj_log_f, sep=' ', header=None, names = log_header, index_col=False)

    #condition selection from subject log
    for fearType in ['Social', 'Spiders', 'Heights']:
        print('\n'+fearType)

        #condition selection from subject log
        subj_log = subj_log_complete[subj_log_complete['Video_name'].str.contains(fearType)]
        subj_log = subj_log.reset_index()
        num_vid = len(subj_log)
    
        #loop through all voxels
        for mask_coordinate in mask_size:
            #     start_time = time.time()
            print('\rworking on '+str(mask_coordinate) + ' out of '+str(max(mask_size)) + ' voxels, ' +str((mask_coordinate)/max(mask_size)*100) + '% done', end="")

            #get the coordinates of the centered voxel
            x = mask_location[0][mask_coordinate]
            y = mask_location[1][mask_coordinate]
            z = mask_location[2][mask_coordinate]

            #get the range of coordinates based on the edge size vox
            x_min = x-vox; y_min = y-vox; z_min = z-vox
            x_max = x+vox+1; y_max = y+vox+1; z_max = z+vox+1;
    #         if x-vox < 0: x_min = 0
    #         if y-vox < 0: y_min = 0
    #         if z-vox < 0: z_min = 0
    #         if x+vox+1 > subj_data_img.shape[0]: x_max = subj_data_img.shape[0]
    #         if y+vox+1 > subj_data_img.shape[1]: y_max = subj_data_img.shape[1]
    #         if z+vox+1 > subj_data_img.shape[2]: z_max = subj_data_img.shape[2]
            
            #calculate ITI period average
            #get averanged run 1 rest data
            rest_mean_run1_temp = []
            for n in list(range(0,int(num_vid/2)-1)):
                #add TR offset to find time for ITI period
                subj_run1_rest_data = subj_run1_data[x_min:x_max,y_min:y_max,z_min:z_max,closest(TR_series, (subj_log['Valence_rating_onset'][n]+TR_BOLD_offset+(TR_mean_rating/3)*TR)): closest(TR_series, subj_log['video_onset'][n+1]+TR_BOLD_offset)]
                subj_run1_rest_data[subj_run1_rest_data==0.]=np.nan
                rest_mean_run1_temp.append(np.nanmean(subj_run1_rest_data)) 
            rest_mean_run1 = np.nanmean(rest_mean_run1_temp)
            #get averanged run 2 rest data
            rest_mean_run2_temp = []
            for n in list(range(int(num_vid/2), num_vid-1)):
                #add TR offset to find time for ITI period
                subj_run2_rest_data = subj_run2_data[x_min:x_max,y_min:y_max,z_min:z_max,closest(TR_series, (subj_log['Valence_rating_onset'][n]+TR_BOLD_offset+(TR_mean_rating/3)*TR)): closest(TR_series, subj_log['video_onset'][n+1]+TR_BOLD_offset)]
                subj_run2_rest_data[subj_run2_rest_data==0.]=np.nan
                rest_mean_run2_temp.append(np.nanmean(subj_run2_rest_data)) 
            rest_mean_run2 = np.nanmean(rest_mean_run2_temp)

            #initialize list for averaged acitivty in mask: vid x TR
            vid_avg_activity_rest_baseline = np.empty((num_vid, TR_video_duration))
            vid_avg_activity_rest_baseline[:] = np.nan
            old_vid = np.empty(num_vid)
            old_vid[:] = np.nan

            #loop through all videos a subject saw
            for vid_num, vid in enumerate(subj_log['Video_name']):
            #     print('vid'+str(vid_num))

                #get old/new info for each video
                if subj_log['novel_vs_familiar(1-N,2-Old)'][vid_num] == 2:
                    old_vid[vid_num] = 1
                elif subj_log['novel_vs_familiar(1-N,2-Old)'][vid_num] == 1:
                    old_vid[vid_num] = 0

                #get video TR start and end
                TR_begin = closest(TR_series, subj_log['video_onset'][vid_num]+TR_BOLD_offset)
                # TR_mean_trial_duration = math.ceil((subj_log['video_offset'][vid_num] - subj_log['video_onset'][vid_num])/TR)
                TR_end = TR_begin+TR_video_duration

                #get video data from nearest TR start
                if subj_log['run_number'][vid_num] == 1:
                    vid_data = subj_run1_data[x_min:x_max,y_min:y_max,z_min:z_max,TR_begin:TR_end]
                    #vid_data.shape
                elif subj_log['run_number'][vid_num] == 2:
                    vid_data = subj_run2_data[x_min:x_max,y_min:y_max,z_min:z_max,TR_begin:TR_end]
                vid_data[vid_data==0.]=np.nan

                #get average of activity in masked area at each slice
                for i in range(vid_data.shape[3]):
                #     #raw BOLD signal mean
                #     vid_avg_activity[vid_num, i] = np.nanmean(vid_data)
                    if subj_log['run_number'][vid_num] == 1:
                        #BOLD signal mean with run1 rest period as baseline
                        vid_avg_activity_rest_baseline[vid_num, i] = np.nanmean(vid_data[:,:,:,i])-rest_mean_run1
                    elif subj_log['run_number'][vid_num] == 2:
                        #BOLD signal mean with run2 rest period as baseline
                        vid_avg_activity_rest_baseline[vid_num, i] = np.nanmean(vid_data[:,:,:,i])-rest_mean_run2

            #get the old/new video timecourse seperately 
            vid_avg_activity_rest_baseline_old = vid_avg_activity_rest_baseline[old_vid==1, :]
            vid_avg_activity_rest_baseline_new = vid_avg_activity_rest_baseline[old_vid==0, :]

            #get autocorrelation for timecourse for average, new, and old images
            correlation_coe = np.corrcoef(np.transpose(vid_avg_activity_rest_baseline))
            correlation_coe[correlation_coe>=0.9999999999]=np.nan
            correlation_coe_mean = np.nanmean(correlation_coe)
            result_template_brain[x,y,z] = correlation_coe_mean
            result_img = nib.Nifti1Image(result_template_brain, subj_data_img.affine, subj_data_img.header)
            nib.save(result_img, save_dir+subj+'_corr_avg_'+fearType+'.nii.gz')

            #new
            correlation_coe_new = np.corrcoef(np.transpose(vid_avg_activity_rest_baseline_new))
            correlation_coe_new[correlation_coe_new>=0.9999999999]=np.nan
            correlation_coe_new_mean = np.nanmean(correlation_coe_new)
            result_template_brain_new[x,y,z] = correlation_coe_new_mean
            result_img_new = nib.Nifti1Image(result_template_brain_new, subj_data_img.affine, subj_data_img.header)
            nib.save(result_img_new, save_dir+subj+'_corr_new_'+fearType+'.nii.gz')

            #old
            correlation_coe_old = np.corrcoef(np.transpose(vid_avg_activity_rest_baseline_old))
            correlation_coe_old[correlation_coe_old>=0.9999999999]=np.nan
            correlation_coe_old_mean = np.nanmean(correlation_coe_old)
            result_template_brain_old[x,y,z] = correlation_coe_old_mean
            result_img_old = nib.Nifti1Image(result_template_brain_old, subj_data_img.affine, subj_data_img.header)
            nib.save(result_img_old, save_dir+subj+'_corr_old_'+fearType+'.nii.gz')

        #     #get corr difference
        #     correlation_coe_NEWvOLD = correlation_coe_new-correlation_coe_old
        #     correlation_coe_mean = np.nanmean(correlation_coe_NEWvOLD)
        #     #map the corr diff result to brain template
        #     result_template_brain[x,y,z] = correlation_coe_mean
        #     print('corr diff: '+str(correlation_coe_mean))
        #     result_img = nib.Nifti1Image(result_template_brain, subj_data_img.affine, subj_data_img.header)
        #     nib.save(result_img, '/home/chendanl/wrkdir/ISECmem_timecourse_searchlight_correlation_analysis/'+subj+'_corr_diff.nii.gz')

        #     elapsed_time = time.time() - start_time
        #     print(elapsed_time)

    # # plotting.view_img('/home/chendanl/wrkdir/ISECmem_timecourse_searchlight_correlation_analysis/'+subj+'_corr_diff.nii.gz',cut_coordsNone=[10,38,35])
    # plotting.view_img('/home/chendanl/wrkdir/ISECmem_timecourse_searchlight_correlation_analysis/'+subj+'_corr_diff.nii.gz')



16: sub_118

Social
working on 658 out of 166077 voxels, 0.3962017618333665% done

/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:78: RuntimeWarning: Mean of empty slice
/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:79: RuntimeWarning: Mean of empty slice
/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:119: RuntimeWarning: Mean of empty slice
/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in greater_equal
/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:128: RuntimeWarning: Mean of empty slice
/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:135: RuntimeWarning: invalid value encountered in greater_equal
/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/

working on 689 out of 166077 voxels, 0.4148678022844825% done

/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:70: RuntimeWarning: Mean of empty slice
/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:71: RuntimeWarning: Mean of empty slice
/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:116: RuntimeWarning: Mean of empty slice


working on 166077 out of 166077 voxels, 100.0% done
Spiders
working on 166077 out of 166077 voxels, 100.0% done
Heights
working on 166077 out of 166077 voxels, 100.0% done

In [6]:

# #plot radius mask sample

# mask_coordinate = mask_size[72000]
# #     start_time = time.time()

# #get the coordinates of the centered voxel
# x = mask_location[0][mask_coordinate]
# y = mask_location[1][mask_coordinate]
# z = mask_location[2][mask_coordinate]

# #get the range of coordinates based on the edge size vox
# x_min = x-vox; y_min = y-vox; z_min = z-vox
# x_max = x+vox+1; y_max = y+vox+1; z_max = z+vox+1;
        
# x=subj_run1_data[:,:,:,0]
# x[:]=np.nan
# x[x_min:x_max,y_min:y_max,z_min:z_max]=1
# x[mask_img.get_fdata()==0]=np.nan

# radius_mask_img = nib.Nifti1Image(x, subj_data_img.affine, subj_data_img.header)
# nib.save(radius_mask_img, save_dir+'radius_mask_img.nii.gz')
# plotting.view_img(save_dir+'radius_mask_img.nii.gz',cmap='rainbow',bg_img=mask_img)
   